# Modules

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import os
import csv
import random
import requests
import time

import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time
import random
from fake_useragent import UserAgent

from datetime import datetime, date
import re
from collections import defaultdict
# from selenium_respectful import RespectfulWebdriver

# from multiprocessing import Pool
import numpy as np
import openpyxl

In [2]:
def make_full_excel_path(tiker):
    basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
    full_path = basic_folder +'\\' + tiker
    return full_path

## Selenium Options

In [3]:
def selelenium(tiker=''):
    basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
    full_path = make_full_excel_path(tiker)

    user_agent = UserAgent()

    option = webdriver.FirefoxOptions()
    option.set_preference('dom.webdriver.enabled', False)
    option.add_argument(f'user-agent={user_agent.random}')
    option.set_preference('dom.webnotifications.enebled', False)
    option.set_preference('media.volume_scale', '0.0')

    option.set_preference("browser.download.folderList", 2)
    option.set_preference("browser.download.manager.showWhenStarting", False)

    option.set_preference("browser.download.dir", full_path)
#     option.set_preference("browser.helperApps.neverAsk.saveToDisk","text/xlsx")
    option.set_preference('browser.helperApps.neverAsk.saveToDisk',
                          'text/plain, application/vnd.ms-excel, text/csv, text/comma-separated-values, application/octet-stream, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    option.headless = False  # True - скрывает отображение работы браузера

    return webdriver.Firefox(executable_path=r"C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\2. Coding\geckodriver.exe", options=option)

In [4]:
def RST():

    return time.sleep(random.randint(3, 6))

# Search Reports_options
### Output = Launch download excels Reports
1. find_cik(tiker)
2. find_all_reports(CIK,periods=20)
3. def convert_from_xmls(xml_report_urls, tiker) 
4. def main_func(tiker)
basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports' (WHO R U?)
5.def open_folder_with_excels(tiker):

In [5]:
def find_cik(tiker):  # '1013934'
    browser = selelenium()

    browser.get('https://sec.report/CIK/')
    time.sleep(3)
    browser.find_element_by_xpath(
        '/html/body/div/div/table[2]/tbody/tr[2]/td[1]/input').send_keys(tiker)
    time.sleep(3)
    browser.find_element_by_xpath(
        '/html/body/div/div/table[2]/tbody/tr[2]/td[2]/input').click()
    time.sleep(3)
    cik = browser.find_elements_by_tag_name("h2")[0].text
    browser.close()
    browser.quit()
    return cik[len(cik) - 7:]

In [6]:
def find_all_reports(CIK,periods=16): # сслыки на отчетности ['https://www.sec.gov/ix?doc=/Archives/edgar/data/0001013934/000101393421000008/stra-20210331.htm',...]
    xml_report_urls = []
    
    browser = selelenium()
    browser.get(f'https://www.sec.gov/edgar/browse/?CIK={CIK}&owner=exclude')
    RST()
    browser.find_element_by_xpath(r'/html/body/main/div[4]/div[2]/div[3]').click()
    
    time.sleep(3)
    browser.find_element_by_xpath(r'/html/body/main/div[4]/div[2]/div[3]/div/button[1]').click()
    
    time.sleep(3)
    how_much_pages = periods
    count_page = 0 
    for page in range(how_much_pages):
        count_page+=1

        browser.find_element_by_xpath(f'/html/body/main/div[5]/div/div[3]/div[3]/div[2]/table/tbody/tr[{count_page}]/td[2]/div/a[1]').click()
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(3)
        xml_report_urls.append(browser.current_url)
        time.sleep(3)
        browser.switch_to.window(browser.window_handles[0])

    return xml_report_urls

In [7]:
def recod_folder(tiker):

    basic_folder = r'C:\Users\nickl\OneDrive\Рабочий стол\Кайросы\1. Market_View\Excel_Reports'
    full_path = basic_folder + '\\' + tiker
    try:

        os.mkdir(full_path)
    except:
        pass
    finally:
        pass
    return full_path

In [8]:
def find_wrong_value(current_url):
    find_wrog_things = current_url.split('/')
    clear_url = find_wrog_things[-2]
    if len(clear_url)>8:
        return current_url
    else:

        find_wrog_things.remove(find_wrog_things[-2])
        return '/'.join(find_wrog_things)


In [9]:
def convert_from_xmls(
    xml_report_urls, tiker
):  # pure urls ['https://www.sec.gov/Archives/edgar/data/0001013934/000101393421000008',
    recod_folder(tiker)
    reports_urls = []
    for urls in xml_report_urls:
        browser = selelenium(tiker)
        browser.implicitly_wait(3)
        
        
        bad_text = (urls[:77].replace(r'ix?doc=/', "") +
                    '/Financial_Report.xlsx')
        
        name_excel_url = find_wrong_value(bad_text)
        print(name_excel_url)   
        
        
        browser.get(name_excel_url)

       

    return reports_urls

In [10]:
def main_func(tiker):
    full_path = make_full_excel_path(tiker)
    try:
        books_list = [os.path.join(full_path, filename) for filename in os.listdir(full_path)]
        if len(books_list)==0:
            recod_folder(tiker) # функция для создания папки каталога excels
            cik_number = find_cik(tiker)
            found_all_reports = find_all_reports(cik_number)
            recod_folder(tiker)
            convert_from_xmls(found_all_reports,tiker)
    except:
        recod_folder(tiker) # функция для создания папки каталога excels
        cik_number = find_cik(tiker)
        found_all_reports = find_all_reports(cik_number)
        recod_folder(tiker)
        convert_from_xmls(found_all_reports,tiker)
#     recod_folder()
    
        books_list = [os.path.join(full_path, filename) for filename in os.listdir(full_path)]
    finally:
        return books_list

# Choose tiker

In [11]:
Tiker = 'OI'
main_func(Tiker)
# https://www.sec.gov/Archives/edgar/data/0000768835/000076883519000021/big-201/Financial_Report.xlsx
# Need to delete /big-201/

['C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(1).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(10).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(11).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(12).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(13).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(14).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(15).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(2).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel

# Excels docs

In [12]:
def open_folder_with_excels(tiker):
    try:
        full_path = make_full_excel_path(tiker)
        
        books_list = list([os.path.join(full_path, filename) for filename in os.listdir(full_path)])
    
        return books_list
    except:
        return main_func(tiker)

In [13]:
open_folder_with_excels(Tiker)

['C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(1).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(10).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(11).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(12).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(13).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(14).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(15).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel_Reports\\OI\\Financial_Report(2).xlsx',
 'C:\\Users\\nickl\\OneDrive\\Рабочий стол\\Кайросы\\1. Market_View\\Excel

# Selection Reports

In [15]:
def check_threee_moth_ended(data_frame):
    
    count_ended = {}
    check_columns = []
    have_found =0 
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if re.search('Months Ended',i):
            count_ended[indexx] = i
            have_found +=1
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if have_found>0:
            data_frame.set_index
        
            try:
                
                new_columns = data_frame.columns[indexx] + ' ' +  second_row
 
            except:
                new_columns = data_frame.columns[indexx]

            check_columns.append(new_columns)

        else:
            
            check_columns.append(i)
            
    data_frame.columns  = check_columns
    if have_found >0:
        data_frame = data_frame.iloc[1:]

    else:
        pass

    
    return data_frame#.iloc[:,1:]

In [16]:
%%time

excl_add = 'Condensed'
exclude_report_list_before_join = ['Net Income per Share',
                       'Summary of Significant Accounting Policies','Common Stock'] 
exclude_report_list = '|'.join(exclude_report_list_before_join)
excludeded_keys = []

data_dict = defaultdict(list)   
REALY_NAMES =  {}
files = open_folder_with_excels(Tiker)
for period in files:
    
#     file = open_folder_with_excels(Tiker)[2]  # cycle 
    wb = openpyxl.load_workbook(period)
    sheetnames_list = wb.sheetnames

    for sheetname in sheetnames_list:
        sheet = wb[sheetname]

        if len(str(sheet['b4'].value))<9 and len(str(sheet['b3'].value))<9:

            header = sheet['A1'].value
        
    #exclude header

            if re.search(exclude_report_list,header):
                excludeded_keys.append(header)
                pass
            else :
                if re.search(excl_add,header):
                    
                    spisok = header.split(' ')
                    spisok.remove(excl_add)
                # (spisok.index(keysss))
                    header=' '.join(spisok)
                else:
                    pass

                REALY_NAMES[header] = sheetname
                a = check_threee_moth_ended(pd.read_excel(period,sheet_name=sheetname))
                
                data_dict.setdefault(header, []).append(a)

# print('OUT' ,len(excludeded_reports))
# for b,i in enumerate(excludeded_reports):
#     print(b+1,i)
# print('______________')

Wall time: 47.8 s


# Look at all selected Reports

In [17]:
print('OUTPUT',len(data_dict),'|||', 'INPUT',len(wb.sheetnames))   
for b,i in enumerate(REALY_NAMES.keys()):
    print(b+1,i)

OUTPUT 152 ||| INPUT 62
1 Document and Entity Information
2 CONDENSED CONSOLIDATED RESULTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions
3 CONDENSED CONSOLIDATED COMPREHENSIVE INCOME (LOSS) - USD ($) $ in Millions
4 CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions
5 CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($) $ in Millions
6 CONDENSED CONSOLIDATED CASH FLOWS - USD ($) $ in Millions
7 Pension Benefit Plans (Tables)
8 Segment Information (Details) - segment
9 Segment Information - Reportable Segments (Details) - USD ($) $ in Millions
10 Segment Information - Total Assets (Details) - USD ($) $ in Millions
11 Revenue (Details) - USD ($) $ in Millions
12 Credit Losses (Details) - USD ($) $ in Millions
13 Inventories (Details) - USD ($) $ in Millions
14 Derivative Instruments - Derivatives and Hedges (Details) - Cash Flow Hedges - USD ($) $ in Millions
15 Derivative Instruments - Balance Sheet Classification (Details) € in Millions, $ in Millions

In [18]:
def check_threee_moth_ended(data_frame):
    
    count_ended = {}
    check_columns = []
    have_found =0 
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if re.search('Months Ended',i):
            count_ended[indexx] = i
            have_found +=1
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if have_found>0:
            data_frame.set_index
        
            try:
                
                new_columns = data_frame.columns[indexx] + ' ' +  second_row
 
#                 check_columns.append(new_columns)
            except:
                new_columns = data_frame.columns[indexx]

            check_columns.append(new_columns)

        else:
            
            check_columns.append(i)
            
    data_frame.columns  = check_columns
    if have_found >0:
        data_frame = data_frame.iloc[1:]

    else:
        pass

    
    return data_frame

# Concat Tables

In [19]:
"Consolidated Statements of Shareholders' Equity (Unaudited) - USD ($) shares in Thousands, $ in Thousands"

"Consolidated Statements of Shareholders' Equity (Unaudited) - USD ($) shares in Thousands, $ in Thousands"

In [20]:
[f'{frame_name} {len(data_dict[frame_name])}' for frame_name in REALY_NAMES.keys() if len(data_dict[frame_name])>10]

['CONDENSED CONSOLIDATED BALANCE SHEETS - USD ($) $ in Millions 12',
 'CONDENSED CONSOLIDATED BALANCE SHEETS (Parenthetical) - USD ($) $ in Millions 12',
 'CONDENSED CONSOLIDATED CASH FLOWS - USD ($) $ in Millions 12',
 'Pension Benefit Plans (Tables) 11',
 'Segment Information - Total Assets (Details) - USD ($) $ in Millions 12',
 'Revenue (Details) - USD ($) $ in Millions 15',
 'Inventories (Details) - USD ($) $ in Millions 16',
 'Derivative Instruments - Balance Sheet Classification (Details) € in Millions, $ in Millions 14',
 'Derivative Instruments - Effects of Derivative Instruments (Details) - USD ($) $ in Millions 12',
 'Debt (Details) $ / shares in Units, € in Millions, $ in Millions 16',
 'Contingencies - Asbestos (Details) 16',
 'Accumulated Other Comprehensive Loss (Details) - USD ($) $ in Millions 11',
 'Earnings Per Share (Details) - USD ($) $ / shares in Units, $ in Millions 16',
 'Supplemental Cash Flow Information (Details) - USD ($) $ in Millions 16',
 'Segment Inform

In [21]:
finder = "Segment Information - Total Assets (Details) - USD ($) $ in Millions"
print(len(data_dict[finder]))
REALY_NAMES[finder]

12


'Segment Information - Total Ass'

In [ ]:
# concat_tables(finder)

# Prepare_Data

In [22]:
from collections import Counter
def convert_column_index_to_unique(frame):
    war_dict = {}
    
    mylist = frame[frame.columns[0]].to_list()


    [war_dict.update({k:0}) for k,v in Counter(mylist).items()]

    pure_list = []
    for name in mylist:
#         name = re.sub('\d','',name)
        if name not in war_dict.keys():
            
            pure_list.append(name)
        else:
            
            war_dict[name] = war_dict[name]+1
            
            new_name = f'{name}{str(war_dict[name])}'
            pure_list.append(new_name.replace('  ',' '))
    return pure_list

In [23]:
def convert_to_finance_period_x(period_input):
    try:
        perods_dict = {'Nov':'Q_1 - 12','Oct':'Q_1 - 11','Sep':'Q_1 - 10','Aug':'Q_2 - 9','Jul':'Q_2 - 8','Jun':'Q_2 - 7','May':'Q_3 - 6','Apr':'Q_3 - 5','Mar':'Q_3 - 4','Feb':'Q_4 - 3','Dec':'Q_4 - 2','Jan':'Q_4 - 1'}
    
#         'Apr', 'Aug', 
        year = re.findall(r'\d\d\d\d',period_input)[0]
        quartal = perods_dict[re.findall(r'\D\D\D',period_input)[0]]
        
    
        return str(year + ' / ' + quartal)
    except:
        return period_input

In [24]:
def exceptions_rows_delete(frame):
    exceptions_rows = [
    'Common Stock$'
    ]
    list_for_delete = []
    for index,row in enumerate(frame.iloc[:,0].to_list()):
    
        for exception_r in exceptions_rows:
    
            if re.match(exception_r,row):
                print(index)
                list_for_delete.append(index)
            
        
        
    return list_for_delete

In [25]:
def cocncat_mean_rows(frame):
#     new_f = frame.fillna(0)
    
    for row in frame.iloc[:,0]:
        if int(re.search('\d',row)[0])>1:
            pass
        
#             print('here')
#     if sum([pd.Series(frame[col]) for col in new_f.columns[1:]])
    
    
    return frame

# For Segments

In [38]:
frame = data_dict[finder][0]
frame.head()
frame.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Segment Information - Total Assets (Details) - USD ($) $ in Millions,Assets,Total assets:,Reportable Segment Totals,Assets,Total assets:,Americas,Assets,Total assets:,Europe,Assets,Total assets:,Asia Pacific,Assets,Total assets:,Other,Assets,Total assets:
"Jun. 30, 2021",NaN,8874.0,NaN,NaN,8474.0,NaN,NaN,4972.0,NaN,NaN,3502.0,NaN,NaN,NaN,NaN,NaN,400.0
"Dec. 31, 2020",NaN,8882.0,NaN,NaN,8434.0,NaN,NaN,4927.0,NaN,NaN,3507.0,NaN,NaN,NaN,NaN,NaN,448.0
"Jun. 30, 2020",NaN,9579.0,NaN,NaN,8661.0,NaN,NaN,4707.0,NaN,NaN,3315.0,NaN,NaN,639.0,NaN,NaN,918.0


In [ ]:
spisok_measure = []


In [132]:
name = []

for i,mes in enumerate(frame.T.columns):
#     print(frame.T.iloc[0:,mes])
# name_name = {}
    if frame.T.iloc[0,frame.T.columns[i]].sum()!=0:
#         print(frame.T.iloc[0,mes])
        first_name = frame.T.iloc[0,mes]
        name.append(first_name)
        if frame.T.iloc[1:,mes-1].sum()==0:
            print(frame.T.iloc[0,mes-1])

TypeError: can only concatenate str (not "int") to str

In [126]:
name

['Total assets:',
 'Total assets:',
 'Total assets:',
 'Total assets:',
 'Total assets:',
 'Total assets:']

In [129]:
frame.T.iloc[0,5]

'Americas'

In [130]:
frame.T.iloc[1:,5].sum()

0

In [26]:

frame = cocncat_mean_rows(data_dict[finder][0])
# df[df.Dates.isnull()]
# [pd.Series(frame.T[col]) for col in frame.T.columns[1:]]
er = frame.T
er.columns = frame.iloc[:,0]
er.drop(labels = [frame.columns[0]],axis = 0)
# new = er.drop(0,axis=0)
# frame[frame.columns[0]] # first_row
# er.columns = er.
avr_list = frame.iloc[:,0]
listaa = []
drop_rows = []
net_sells_dict = {}

net_sells_list = []
for index,row in enumerate(avr_list):
    if re.search('Net sales',row):
        net_sells_list.append(row)
    if re.search('Line Items',row):
        index_for_row_line = index
#         print(row)
        drop_rows.append(index)
#         print(avr_list[index_for_row_line+2])
        try:
            if re.search('Member',avr_list[index_for_row_line+3]): 
#                 print(avr_list[index_for_row_line+3])
#                 withiut_member = re.sub(r'[Member]','',avr_list[index_for_row_line+2])
                accumulated_row_name = f'{avr_list[index_for_row_line+3]}'
                drop_rows.append(index_for_row_line+2)
                net_sells_dict[avr_list[index_for_row_line+2]] = accumulated_row_name
                print(f'{accumulated_row_name} {index}')
                listaa.append(index+2)
          
        except:
            pass
        finally:
            pass
# net_sells_dict[net_sells_list[-1]] = 'Acumulated Sellls'

#         if re.findall('Member',row)[0]

#         index_for_row_line+
# frame.iloc[0,1:].isnull()
#         if (frame.iloc[:,0])[index]==(re.findall('Net sales',row)[0]):
#             print(row)
# #         print('here')
frame[frame.index.isin(listaa)]
frame.iloc[:,0].rename(net_sells_dict,axis=0)
# list(map(lambda x: net_sells_dict[x],frame.iloc[:,0]))
frame[~frame.index.isin(drop_rows)]
net_sells_dict

TypeError: 'NoneType' object is not subscriptable

In [ ]:
frame.head()

In [69]:
# Начинаьт надо с member

# frame

In [133]:
def pop_unneed_numbers(frame):
    
   
    withoud_number_list = [re.sub('\d','',row) for row in frame.iloc[:,0]]
    return withoud_number_list

In [ ]:
cocncat_mean_rows(data_dict[finder][0])

In [134]:
def scrub_one_period(frame):
    frame.iloc[:,0] = pop_unneed_numbers(frame)
    frame.iloc[:,0] = convert_column_index_to_unique(frame) # assigned unque Value for 0 Column
    frame.columns = list(map(lambda date: convert_to_finance_period_x(date),frame.columns)) # convert for normal periods
    
#     w = exceptions_rows_delete(frame)
    return frame

In [135]:
def clear_set(ar):
    ar[ar.columns[0]] = ar[ar.columns[0]].apply(lambda x: x[0:75])
    
    return ar

In [136]:
def clear_frame(frame):
    if len(str(frame.columns[0]))<4 :
        print('here')   
        frame.index=frame.iloc[:,0]
        frame.index.name = 'Category'
        frame.drop(frame.columns[[0]],axis='columns',inplace=True)
    else:
        pass
    return frame
    

In [137]:
def concat_tables(key_name):
    frame_size = len(data_dict[key_name])-1
    structure = list(map(lambda dirty_drame:scrub_one_period(dirty_drame),data_dict[key_name]))
    def recurs(x):
       
        if x ==0:
            col_name = structure[x].columns[0]
            return (structure[x]).merge((structure[x+1]),on=key_name,how='outer',suffixes=('', '_y'))
        
            
        else:
            col_name = structure[x].columns[0]
            
            return recurs(x-1).merge((structure[x-1]),on=key_name,how='outer',suffixes=('', '_y'))
        
    frame = recurs(frame_size)
    
    try:
        
        if frame_size==1:
            return (structure[0])
        else:
            frame = recurs(frame_size)
            frame.drop(frame.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        
            return frame
    except Exception as ex:
        return ex
    


# GO

In [141]:
conc = concat_tables(finder)
new = conc[conc.columns[1:].sort_values(ascending=True)]
new.insert(0,conc.columns[0],conc[conc.columns[0]])

In [142]:
new

,Segment Information - Total Assets (Details) - USD ($) $ in Millions,2017 / Q_1 - 10,2017 / Q_2 - 7,2017 / Q_3 - 4,2017 / Q_4 - 2,2018 / Q_1 - 10,2018 / Q_2 - 7,2018 / Q_3 - 4,2018 / Q_4 - 2,2019 / Q_1 - 10,2019 / Q_2 - 7,2019 / Q_3 - 4,2019 / Q_4 - 2,2020 / Q_1 - 10,2020 / Q_2 - 7,2020 / Q_3 - 4,2020 / Q_4 - 2,2021 / Q_2 - 7,2021 / Q_3 - 4
0,Assets1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total assets:1,9999.0,9780.0,9459.0,9756.0,9940.0,9818.0,10281.0,9699.0,9548.0,10723.0,10152.0,9610.0,8624.0,9579.0,9504.0,8882.0,8874.0,8825.0
2,Reportable Segment Totals1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Assets2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Total assets:2,9784.0,9572.0,9289.0,9545.0,9674.0,9615.0,10080.0,9451.0,8989.0,10381.0,9906.0,9085.0,7970.0,8661.0,8591.0,8434.0,8474.0,8291.0
5,Americas1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Assets3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Total assets:3,5598.0,5511.0,5433.0,5411.0,5455.0,5402.0,5637.0,5497.0,5241.0,5994.0,5621.0,5264.0,4663.0,4707.0,4693.0,4927.0,4972.0,4755.0
8,Europe1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Assets4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
